<h1><center>ЛОГИРУЕМ МОДЕЛЬ</h1></center>

Пора продолжить работу над проблемой оттока клиентов! После первой итерации вы получили базовое решение — модель логистической регрессии. Вы также провели свой первый эксперимент и залогировали основные артефакты и сам набор данных.
<br><br>Прежде чем перейти к логированию модели в MLflow, важно убедиться, что вам доступна сама модель, а MLflow настроен в соответствии с предыдущими уроками. 
<br><br>
1.(**команда sh run_mlflow_server.sh делает все шаги первого этапа**) Поднимите MLflow, используя предложенный ранее вариант конфигурации:

- локальный Tracking Server на виртуальной машине,
- удалённые хранилище экспериментов и хранилище артефактов.

Не забудьте настроить соответствующие переменные окружения для подключения к базе данных Postgres и объектному хранилищу S3.

2.(**код в этом блокноте**) Определите необходимые глобальные переменные. Одной из них будет EXPERIMENT_NAME, которую вы уже использовали. Вторую переменную, RUN_NAME, вы можете назвать самостоятельно, чтобы ясно отразить суть запуска.


In [1]:
import pandas as pd
import numpy as np
import random
import os
import joblib

import mlflow
from mlflow.tracking import MlflowClient

import catboost as cb
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

from category_encoders import CatBoostEncoder

In [3]:
# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJE3Nlz8iDILW5VTYM1ihQB' # os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44' # os.getenv("AWS_SECRET_ACCESS_KEY") #  получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

YOUR_NAME = "unique_experiment-8" # введите своё имя для создания уникального эксперимента
assert YOUR_NAME, "alexnikzotov"

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = f"test_connection_experiment_{YOUR_NAME}"
RUN_NAME = "test_connection_run"

# тестовые данные
METRIC_NAME = "test_metric"
METRIC_VALUE = 0

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# создаём тестовый эксперимент и записываем в него тестовую информацию
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metric(METRIC_NAME, METRIC_VALUE) 

3.Для логирования используйте либо уже имеющуюся модель, полученную в первом спринте, либо обучите новую модель по своему усмотрению - **обучим новую модель**. Загрузим и подготовим данные:


In [4]:
df = pd.read_csv('users_churn.csv')
print('Колво строк до обработки:',df.shape[0])

# Закодируем целевую переменную
df['target'] = (df['end_date'].notna()).astype(int)

# Выделим признаки в три отдельные таблицы для дальнейшей работы:
features = df.drop(['customer_id','target'],axis=1)
num_features = features.select_dtypes(include=['float', 'int'])
date_features = features.select_dtypes(include='datetime64[ns]')
cat_features = features.select_dtypes(include='object')

# Посчитаем колво уникальных значений для катег. переменных и создадим создадим два датасета:
unique_values_per_col = cat_features.nunique().value_counts()
binary_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()==2] ]
other_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()!=2] ]

# Бинарные подразделяем на два - "да/нет" и другие бинарные:
yes_no_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()==True] ]
other_binary_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()!=True] ]

# Дубликаты
is_duplicated_id = df.duplicated(subset=['customer_id'], keep=False)

# Пропуски
cols_with_nans = df.isnull().sum()
cols_with_nans = cols_with_nans[cols_with_nans > 0].index.drop('end_date')
for col in cols_with_nans:
    if df[col].dtype in [float, int]:
        fill_value = df[col].mean()
    elif df[col].dtype == 'object':
        fill_value = df[col].mode().iloc[0]
    df[col] = df[col].fillna(fill_value)

# Выбросы
num_cols = df.select_dtypes(['float']).columns
threshold = 1.5
potential_outliers = pd.DataFrame()
for col in num_cols:
	Q1 = df[col].quantile(0.25)
	Q3 = df[col].quantile(0.75)
	IQR = Q3 - Q1
	margin = threshold * IQR
	lower = Q1 - margin
	upper = Q3 + margin
	potential_outliers[col] = ~df[col].between(lower, upper)
outliers = potential_outliers.any(axis=1)

df.drop(columns=['id', 'customer_id', 'begin_date', 'end_date'], inplace=True)
df[-3:]


Колво строк до обработки: 7019


,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
7016,Month-to-month,Yes,Electronic check,29.60,346.45,DSL,Yes,No,No,No,No,No,Female,0,Yes,Yes,No,0
7017,Month-to-month,Yes,Mailed check,74.40,306.60,Fiber optic,No,No,No,No,No,No,Male,1,Yes,No,Yes,1
7018,Two year,Yes,Bank transfer (automatic),105.65,6844.50,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Male,0,No,No,No,0


Обучим модель:

In [5]:

# Разделим данные на две части - для обучения и для проверки качества предсказания:
X_tr, X_val, y_tr, y_val = train_test_split(df, df['target'], stratify=df['target']) 

# Тренировочная выборка
cat_features_tr = X_tr.select_dtypes(include='object')
potential_binary_features_tr = cat_features_tr.nunique() == 2

binary_cat_features_tr = cat_features_tr[potential_binary_features_tr[potential_binary_features_tr].index]
other_cat_features_tr = cat_features_tr[potential_binary_features_tr[~potential_binary_features_tr].index]
num_features_tr = X_tr.select_dtypes(['float'])

# Валидационная выборка
cat_features_val = X_val.select_dtypes(include='object')
potential_binary_features_val = cat_features_val.nunique() == 2

binary_cat_features_val = cat_features_val[potential_binary_features_val[potential_binary_features_val].index]
other_cat_features_val = cat_features_val[potential_binary_features_val[~potential_binary_features_val].index]
num_features_val = X_val.select_dtypes(['float'])

binary_cols = binary_cat_features_tr.columns.tolist()
non_binary_cat_cols = other_cat_features_tr.columns.tolist()
num_cols = num_features_tr.columns.tolist()

# Определим список трансформаций в рамках ColumnTransformer
preprocessor = ColumnTransformer( [ ('binary', OneHotEncoder(drop='if_binary'), binary_cols),
                                    ('cat', CatBoostEncoder(), non_binary_cat_cols),
                                    ('num', StandardScaler(), num_cols) ],verbose_feature_names_out=False )

# Трансформируем исходные данные с помощью созданного preprocessor
X_tr_transformed = preprocessor.fit_transform(X_tr, y_tr)
X_val_transformed = preprocessor.transform(X_val)

# Создадим модель:
model = CatBoostClassifier()

# Обучим модель:
model.fit(X_tr_transformed, y_tr)

Learning rate set to 0.020938
0:	learn: 0.6790830	total: 63.5ms	remaining: 1m 3s
1:	learn: 0.6663198	total: 66.3ms	remaining: 33.1s
2:	learn: 0.6534113	total: 69.2ms	remaining: 23s
3:	learn: 0.6423960	total: 72ms	remaining: 17.9s
4:	learn: 0.6314138	total: 74.6ms	remaining: 14.8s
5:	learn: 0.6209857	total: 77.2ms	remaining: 12.8s
6:	learn: 0.6106950	total: 79.9ms	remaining: 11.3s
7:	learn: 0.6011115	total: 82.5ms	remaining: 10.2s
8:	learn: 0.5919951	total: 85.2ms	remaining: 9.38s
9:	learn: 0.5841230	total: 87.8ms	remaining: 8.69s
10:	learn: 0.5755402	total: 90.3ms	remaining: 8.12s
11:	learn: 0.5670845	total: 92.9ms	remaining: 7.65s
12:	learn: 0.5589549	total: 95.6ms	remaining: 7.25s
13:	learn: 0.5526662	total: 98.4ms	remaining: 6.93s
14:	learn: 0.5463947	total: 102ms	remaining: 6.67s
15:	learn: 0.5396232	total: 104ms	remaining: 6.41s
16:	learn: 0.5338411	total: 107ms	remaining: 6.19s
17:	learn: 0.5283129	total: 110ms	remaining: 5.98s
18:	learn: 0.5233864	total: 112ms	remaining: 5.8s
19

Проверим качество нашей модели:

In [6]:
# Сделаем предсказание:
y_val_pred = model.predict(X_val_transformed)

# Определим точность:
accuracy = str(accuracy_score(y_val, y_val_pred))[:4]
print(f"Точность модели: {accuracy}")

Точность модели: 0.78


In [7]:
# Базовый код, который нужно будет использовать после обучения
experiment_id = '4'
with mlflow.start_run(run_name='log_model', experiment_id=experiment_id) as run:
    run_id = run.info.run_id
  
    model_info = mlflow.catboost.log_model(cb_model=model, artifact_path="models") 

4.Установите библиотеку scikit-learn в ваше окружение: pip install scikit-learn==1.3.1.



In [ ]:
# pip install scikit-learn==1.3.1.


Также для успешного выполнения заданий подготовьте следующие пререквизиты:
- Объект модели. Это та самая обученная модель, которую вы планируете логировать с помощью MLflow. Сохраните её в переменную model.
- Предсказанные моделью значения. Полученные прогнозы модели, включая вероятности положительного исхода и бинарные значения, которые вы определили. - Сохраните эти значения в переменные: prediction — бинарные значения 0 или 1, proba — вероятности.
- Истинные метки и данные для предсказания. Первое — это истинные значения целевой переменной для вашего тестового набора данных. Сохраните их в переменную y_test. Второе — датасет с признаками для модели, который уже сохранён в переменную X_test.

In [8]:
# Получение бинарных значений:
prediction = model.predict(X_val_transformed)

# Получение вероятностей положительного исхода:
proba = model.predict_proba(X_val_transformed)[:, 1]

# Истиные метки:
y_val

# Данные для предсказания (Признаки для тестовой выборки):
X_val


,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
2850,Two year,Yes,Credit card (automatic),88.55,6306.50,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Male,1,Yes,No,Yes,0
4298,Two year,No,Credit card (automatic),82.10,2603.10,DSL,No,Yes,Yes,Yes,Yes,Yes,Male,0,Yes,Yes,No,0
3843,Month-to-month,No,Bank transfer (automatic),20.65,93.55,Fiber optic,No,No,No,No,No,No,Male,0,No,No,No,0
3575,Month-to-month,Yes,Credit card (automatic),99.45,919.40,Fiber optic,No,Yes,No,No,Yes,Yes,Male,1,No,No,Yes,1
5271,Month-to-month,Yes,Electronic check,79.65,79.65,Fiber optic,No,No,No,No,Yes,No,Female,0,No,No,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4253,Two year,Yes,Bank transfer (automatic),91.15,6637.90,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Female,0,Yes,No,Yes,0
1420,One year,No,Mailed check,19.95,936.70,Fiber optic,No,No,No,No,No,No,Female,0,No,Yes,No,0
2612,Month-to-month,Yes,Electronic check,68.35,1299.80,DSL,Yes,No,Yes,Yes,Yes,No,Male,0,No,No,No,0
891,Month-to-month,Yes,Credit card (automatic),94.30,2679.70,Fiber optic,Yes,Yes,No,No,No,Yes,Male,0,No,No,Yes,0
